In [4]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import xgboost as xgb
from rich.console import Console
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error

ROOT_DIR = os.path.dirname(
os.path.dirname(os.path.abspath('')))


sys.path.append(os.path.join(ROOT_DIR, "code"))
from tools.data_loader import TestSet, TestSplit, data_loader
from tools.save import save_as_baseline, save_datasets, save_models
from tools.train import evaluate_models, print_test_samples, train_models, print_problematic_samples
from tools.utils import StructureEncoding, Target, check_xgboost_gpu

# Define global variables
DATA_DIR = os.path.join(ROOT_DIR, "data/")

DATA_PATH = os.path.join(DATA_DIR, "data.csv")


In [7]:
console = Console(record=True)
prompt_user = False

encoding = StructureEncoding.ATOMIC

console.log(f"[bold green]Started pipeline for {encoding}")
target = Target.DELTA_E
test_sets_cfg = [
    TestSet("Parameter gen.", size=0.1, split=TestSplit.ROW),
    TestSet("Structure gen.", size=0.1, split=TestSplit.STRUCTURE),
]

# Data Loading
X_train, y_train, test_sets = data_loader(
    target=target,
    encoding=encoding,
    data_path=DATA_PATH,
    test_sets_cfg=test_sets_cfg,
    console=console,
    remove_ref_rows=True,
)

[10:00:42] Started pipeline for StructureEncoding.ATOMIC                      ]8;id=319358;file:///var/folders/zv/j2vc13hx0x72z8g7wvs60p3w0000gn/T/ipykernel_27577/2179649547.py\2179649547.py]8;;\:]8;id=207849;file:///var/folders/zv/j2vc13hx0x72z8g7wvs60p3w0000gn/T/ipykernel_27577/2179649547.py#6\6]8;;\

Output()

KeyboardInterrupt: 

In [3]:
base = 10
def magnitude(x):
    return int(np.ceil(np.log(x) / np.log(base)))

def magnitude_inv(x):
    return base**float(x-1)
    # for the regression transformation, taking the prediction minus 1 yields way better MAPE scores
    # I think that's because predicting a result with an order of magnitude superior to the actual value
    # yields an APE error of 10 (1000%) while predicting a result with an order of magnitude inferior to the actual value
    # yields an APE error of at most 1 (100%)

def magnitude_transform(a):
    return np.vectorize(magnitude)(a)

def magnitude_inv_transform(a):
    return np.vectorize(magnitude_inv)(a)

In [4]:
x = 1E-6
magnitude(x)

-5

In [5]:
magnitude_y_train = magnitude_transform(y_train)

In [6]:
print("max magnitude: ", np.max(magnitude_y_train))
print("min magnitude: ", np.min(magnitude_y_train))

max magnitude:  1
min magnitude:  -8


In [7]:
model = RandomForestClassifier() 
model.fit(X_train, magnitude_y_train)

RandomForestClassifier()

In [8]:
for test_name, X_test, y_test in test_sets:
    magnitude_y_pred = model.predict(X_test)
    magnitude_y_test = magnitude_transform(y_test)
    score = accuracy_score(magnitude_y_test, magnitude_y_pred)
    mape = mean_absolute_percentage_error(y_test, magnitude_inv_transform(magnitude_y_pred))
    console.print(f"Accuracy {test_name}: {score}")
    console.print(f"MAPE {test_name}: {mape}")

Accuracy Parameter gen.: 0.9726453908595575

MAPE Parameter gen.: 1.0995079023865462

Accuracy Structure gen.: 0.7105619753829939

MAPE Structure gen.: 11.442420744794015

In [ ]:
model = xgb.XGBClassifier(label_encoder=False)
model.fit(X_train, magnitude_y_train)

/Users/philipp/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:37:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "label_encoder" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:37:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
for test_name, X_test, y_test in test_sets:
    magnitude_y_pred = model.predict(X_test)
    magnitude_y_test = magnitude_transform(y_test)
    score = accuracy_score(magnitude_y_test, magnitude_y_pred)
    mape = mean_absolute_percentage_error(y_test, magnitude_inv_transform(magnitude_y_pred))
    console.print(f"Accuracy {test_name}: {score}")
    console.print(f"MAPE {test_name}: {mape}")

In [ ]:
class RegModel:
    def __init__(self, class_model):
        self.class_model = class_model

    def predict(self, X):
        return magnitude_inv_transform(self.class_model.predict(X))
reg_models = {"RandomForest": RegModel(model)}

In [ ]:
evaluate_models(reg_models, X_train, y_train, test_sets, console)

In [ ]:
print_test_samples(reg_models, test_sets, console)